# Special Education (IDEA)

**Author:** Luke Steuber  
**Dataset:** IDEA Part B Special Education - National and State Data (2022-23)  
**Sources:**
- NCES Digest of Education Statistics 2023
- Civil Rights Data Collection (CRDC) 2020-21
- U.S. Department of Education, Office of Special Education Programs

## Overview

This notebook analyzes special education data under the Individuals with Disabilities Education Act (IDEA). IDEA Part B serves children and youth ages 3-21 with disabilities. We examine:

- **7.5 million students** served (15.2% of public school enrollment) — an all-time high
- **13 disability categories** with specific learning disability (32%) as the largest
- **Autism growth**: 8.5x increase since 2000 (1.5% → 13% of IDEA students)
- **Historical trends**: Enrollment doubled from 3.7M (1976) to 7.5M (2023)
- **Discipline disparities**: 2-3x higher suspension rates for IDEA students
- **State variation**: Pennsylvania (21.1%) vs. Hawaii (11.7%) — nearly 2:1 difference

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")
%matplotlib inline

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries loaded successfully")

## 1. Load Data

In [ ]:
# Load enriched IDEA data
with open('../idea_special_education_enriched.json', 'r') as f:
    idea_data = json.load(f)

# Load CRDC civil rights data
with open('../crdc_disability_data.json', 'r') as f:
    crdc_data = json.load(f)

print(f"Dataset: {idea_data['dataset_name']}")
print(f"School Year: {idea_data['school_year_current']}")
print(f"Total Students Served: {idea_data['national_summary_2022_23']['total_students_served_formatted']}")
print(f"Percent of Enrollment: {idea_data['national_summary_2022_23']['percent_of_public_school_enrollment']}%")

## 2. Disability Categories (2022-23)

IDEA recognizes 13 disability categories. Specific learning disability (SLD) is the largest, followed by speech/language impairment and other health impairment (primarily ADHD).

In [ ]:
# Create DataFrame from disability categories
disability_df = pd.DataFrame(idea_data['disability_categories_2022_23'])
disability_df = disability_df.sort_values('percentage', ascending=False)

print("Top 5 Disability Categories (2022-23):")
print(disability_df[['category', 'count_formatted', 'percentage', 'percent_of_total_enrollment']].head())

In [ ]:
# Visualization 1: Disability Categories Bar Chart
fig, ax = plt.subplots(figsize=(14, 8))

# Sort by percentage for better visualization
disability_sorted = disability_df.sort_values('percentage', ascending=True)

colors = plt.cm.Set3(np.linspace(0, 1, len(disability_sorted)))
bars = ax.barh(disability_sorted['category'], disability_sorted['percentage'], color=colors)

# Add value labels
for i, (idx, row) in enumerate(disability_sorted.iterrows()):
    ax.text(row['percentage'] + 0.5, i, f"{row['percentage']:.1f}%\n({row['count_formatted']})", 
            va='center', fontsize=9)

ax.set_xlabel('Percentage of IDEA Students', fontsize=12, fontweight='bold')
ax.set_title('IDEA Students by Disability Category (2022-23)\n7.5 Million Students (15.2% of Public School Enrollment)', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='x', alpha=0.3)
ax.set_xlim(0, max(disability_sorted['percentage']) * 1.2)

plt.tight_layout()
plt.show()

print("\nKey Findings:")
print("- Specific learning disability (SLD) remains the largest category at 32%")
print("- Top 3 categories (SLD, speech, other health impairment) account for 66% of all IDEA students")
print("- Autism is the 4th largest category at 13% (fastest-growing)")

In [ ]:
# Visualization 2: Disability Categories Pie Chart (Top 7 + Others)
fig, ax = plt.subplots(figsize=(12, 10))

# Group smaller categories
top_categories = disability_df.head(7).copy()
other_count = disability_df.iloc[7:]['count'].sum()
other_pct = disability_df.iloc[7:]['percentage'].sum()

top_categories = pd.concat([top_categories, pd.DataFrame([
    {'category': 'Other (6 categories)', 'count': other_count, 'percentage': other_pct}
])])

colors_pie = plt.cm.Set3(np.linspace(0, 1, len(top_categories)))
wedges, texts, autotexts = ax.pie(
    top_categories['percentage'], 
    labels=top_categories['category'],
    autopct='%1.1f%%',
    startangle=90,
    colors=colors_pie,
    textprops={'fontsize': 10}
)

# Bold the percentage text
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

ax.set_title('Distribution of IDEA Students by Disability (2022-23)', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

## 3. Historical Enrollment Trends (1976-2023)

IDEA enrollment has more than doubled since 1976, growing from 3.7 million to 7.5 million students. As a percentage of total enrollment, it increased from 8.3% to 15.2%.

In [ ]:
# Extract historical trends
trends = idea_data['historical_trends_1976_2023']
total_by_year = trends['total_served_by_year']
pct_by_year = trends['percent_of_total_enrollment_by_year']

# Create DataFrame
years = list(total_by_year.keys())
trend_df = pd.DataFrame({
    'year': years,
    'total_students': list(total_by_year.values()),
    'pct_enrollment': list(pct_by_year.values())
})

# Convert year strings to integers for plotting
trend_df['year_int'] = trend_df['year'].str.split('-').str[0].astype(int)

print("IDEA Enrollment Growth:")
print(trend_df[['year', 'total_students', 'pct_enrollment']].head())
print("...")
print(trend_df[['year', 'total_students', 'pct_enrollment']].tail())

In [ ]:
# Visualization 3: IDEA Enrollment Trends (Dual Axis)
fig, ax1 = plt.subplots(figsize=(14, 7))

# Plot total students (left axis)
color1 = 'tab:blue'
ax1.set_xlabel('School Year', fontsize=12, fontweight='bold')
ax1.set_ylabel('Total Students Served (Millions)', color=color1, fontsize=12, fontweight='bold')
line1 = ax1.plot(trend_df['year_int'], trend_df['total_students'] / 1e6, 
                 color=color1, marker='o', linewidth=2.5, markersize=6, label='Total Students')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.grid(alpha=0.3)

# Plot percentage (right axis)
ax2 = ax1.twinx()
color2 = 'tab:red'
ax2.set_ylabel('% of Public School Enrollment', color=color2, fontsize=12, fontweight='bold')
line2 = ax2.plot(trend_df['year_int'], trend_df['pct_enrollment'], 
                 color=color2, marker='s', linewidth=2.5, markersize=6, label='% of Enrollment')
ax2.tick_params(axis='y', labelcolor=color2)

# Title and annotations
ax1.set_title('IDEA Enrollment Trends (1976-2023)\nFrom 3.7M (8.3%) to 7.5M (15.2%)', 
              fontsize=14, fontweight='bold', pad=20)

# Annotate key points
ax1.annotate('3.7M\n(8.3%)', xy=(1976, 3.694), xytext=(1980, 5.5),
             arrowprops=dict(arrowstyle='->', color='gray', lw=1.5),
             fontsize=10, ha='left')
ax1.annotate('7.5M\n(15.2%)', xy=(2022, 7.526), xytext=(2015, 6.8),
             arrowprops=dict(arrowstyle='->', color='gray', lw=1.5),
             fontsize=10, ha='right')
ax1.annotate('COVID dip\n2020-21', xy=(2020, 7.183), xytext=(2010, 6.2),
             arrowprops=dict(arrowstyle='->', color='gray', lw=1.5),
             fontsize=9, ha='right', color='darkred')

# Rotate x-axis labels
ax1.set_xticks(trend_df['year_int'][::2])  # Show every other year
ax1.set_xticklabels(trend_df['year'][::2], rotation=45, ha='right')

# Legend
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left', fontsize=10)

plt.tight_layout()
plt.show()

print("\nKey Findings:")
print("- 104% growth: 3.7M (1976) → 7.5M (2023)")
print("- As % of enrollment: 8.3% (1976) → 15.2% (2023) — 83% relative increase")
print("- COVID-19 caused brief dip in 2020-21, followed by rapid recovery to all-time high")
print("- Most rapid growth 1976-2005; plateaued 2005-2015; resumed 2015-present")

## 4. Autism Growth Trend (2000-2023)

Autism is the fastest-growing disability category, increasing 8.5x from 1.5% of IDEA students in 2000-01 to 13% in 2022-23.

In [ ]:
# Extract autism trend data
autism_trend = idea_data['disability_category_trends']['autism']
autism_df = pd.DataFrame(autism_trend['percent_of_idea_students'])

# Extract years as integers
autism_df['year_int'] = autism_df['year'].str.split('-').str[0].astype(int)

print("Autism Growth Trend:")
print(autism_df)

In [ ]:
# Visualization 4: Autism Growth (% of IDEA Students)
fig, ax = plt.subplots(figsize=(12, 7))

# Line plot
ax.plot(autism_df['year_int'], autism_df['percentage'], 
        marker='o', linewidth=3, markersize=8, color='tab:purple')

# Fill area under curve
ax.fill_between(autism_df['year_int'], autism_df['percentage'], alpha=0.3, color='tab:purple')

# Labels and title
ax.set_xlabel('School Year (Starting)', fontsize=12, fontweight='bold')
ax.set_ylabel('% of IDEA Students', fontsize=12, fontweight='bold')
ax.set_title('Autism Identification Among IDEA Students (2000-2023)\n8.5x Increase from 1.5% to 13.0%', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(alpha=0.3)

# Annotate start and end
ax.annotate(f"2000-01\n1.5%", xy=(2000, 1.5), xytext=(2002, 3),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
            fontsize=11, ha='left', fontweight='bold')
ax.annotate(f"2022-23\n13.0%", xy=(2022, 13.0), xytext=(2018, 11),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
            fontsize=11, ha='right', fontweight='bold')

# Add growth rate text
ax.text(2011, 8, '8.5x growth\n(8.2% annual avg)', 
        fontsize=12, ha='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

ax.set_ylim(0, 15)

plt.tight_layout()
plt.show()

print("\nKey Findings:")
print("- Autism grew from 1.5% (2000-01) to 13.0% (2022-23) — 8.5x increase")
print("- Average annual growth rate: 8.2% per year")
print("- Now the 4th largest disability category (980,272 students)")
print("- As % of total public school enrollment: 0.2% (2000) → 2.0% (2022)")

## 5. State Variation in IDEA Identification

States vary dramatically in IDEA identification rates, from 11.7% (Hawaii) to 21.1% (Pennsylvania) — nearly a 2:1 difference.

In [ ]:
# Extract state data
state_data = idea_data['state_data_2022_23']
state_df = pd.DataFrame.from_dict(state_data, orient='index')
state_df = state_df.reset_index().rename(columns={'index': 'state'})
state_df = state_df.sort_values('pct_of_enrollment', ascending=False)

print("Top 10 States (Highest % IDEA):")
print(state_df[['state', 'total_served', 'pct_of_enrollment']].head(10))
print("\nBottom 10 States (Lowest % IDEA):")
print(state_df[['state', 'total_served', 'pct_of_enrollment']].tail(10))

In [ ]:
# Visualization 5: State Variation (Top 15 and Bottom 15)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Top 15 states
top15 = state_df.head(15).sort_values('pct_of_enrollment', ascending=True)
colors_top = plt.cm.Greens(np.linspace(0.4, 0.9, len(top15)))
ax1.barh(top15['state'], top15['pct_of_enrollment'], color=colors_top)
ax1.set_xlabel('% of Enrollment', fontsize=11, fontweight='bold')
ax1.set_title('Top 15 States\n(Highest IDEA Identification)', fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
ax1.axvline(15.17, color='red', linestyle='--', linewidth=2, label='National Avg (15.2%)')
ax1.legend(fontsize=9)

# Bottom 15 states
bottom15 = state_df.tail(15).sort_values('pct_of_enrollment', ascending=True)
colors_bottom = plt.cm.Reds(np.linspace(0.4, 0.9, len(bottom15)))
ax2.barh(bottom15['state'], bottom15['pct_of_enrollment'], color=colors_bottom)
ax2.set_xlabel('% of Enrollment', fontsize=11, fontweight='bold')
ax2.set_title('Bottom 15 States\n(Lowest IDEA Identification)', fontsize=12, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)
ax2.axvline(15.17, color='red', linestyle='--', linewidth=2, label='National Avg (15.2%)')
ax2.legend(fontsize=9)

plt.suptitle('State Variation in IDEA Identification Rates (2022-23)', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\nKey Findings:")
print(f"- Highest: Pennsylvania (21.1% of enrollment)")
print(f"- Lowest: Hawaii (11.7% of enrollment)")
print(f"- Range: 9.4 percentage points (nearly 2:1 ratio)")
print(f"- Likely reflects differences in state disability determination practices, funding, and SES levels")

## 6. Discipline Disparities (CRDC 2020-21)

Students with disabilities face significantly higher discipline rates than their non-disabled peers.

In [ ]:
# Extract discipline data from CRDC
discipline = crdc_data['discipline_disparities']
summary = crdc_data['summary_statistics']['discipline']

print("Discipline Rate Comparison (2020-21):")
print(f"In-School Suspension:")
print(f"  IDEA students: {summary['idea_iss_rate_pct']:.2f}%")
print(f"  Non-IDEA students: {summary['nonidea_iss_rate_pct']:.2f}%")
print(f"  Disparity ratio: {summary['idea_iss_rate_pct'] / summary['nonidea_iss_rate_pct']:.1f}x higher")
print(f"\nOut-of-School Suspension:")
print(f"  IDEA students: {summary['idea_oss_rate_pct']:.2f}%")
print(f"  Non-IDEA students: {summary['nonidea_oss_rate_pct']:.2f}%")
print(f"  Disparity ratio: {summary['idea_oss_rate_pct'] / summary['nonidea_oss_rate_pct']:.1f}x higher")

In [ ]:
# Visualization 6: Discipline Disparity Comparison
fig, ax = plt.subplots(figsize=(12, 7))

# Prepare data
categories = ['In-School\nSuspension', 'Out-of-School\nSuspension', 'Law Enforcement\nReferral', 'School-Related\nArrest']
idea_rates = [
    summary['idea_iss_rate_pct'],
    summary['idea_oss_rate_pct'],
    (discipline['IDEA Students Referred to Law Enforcement']['national_total'] / 
     crdc_data['national_overview_2020_21']['total_enrollment']['idea_students']) * 100,
    (discipline['IDEA Students With School Related Arrest']['national_total'] / 
     crdc_data['national_overview_2020_21']['total_enrollment']['idea_students']) * 100
]
nonidea_rates = [
    summary['nonidea_iss_rate_pct'],
    summary['nonidea_oss_rate_pct'],
    (discipline['Non-IDEA Students Referred to Law Enforcement']['national_total'] / 
     crdc_data['national_overview_2020_21']['total_enrollment']['non_disabled_idea']) * 100,
    (discipline['Non-IDEA Students With School Related Arrest']['national_total'] / 
     crdc_data['national_overview_2020_21']['total_enrollment']['non_disabled_idea']) * 100
]

x = np.arange(len(categories))
width = 0.35

bars1 = ax.bar(x - width/2, idea_rates, width, label='IDEA Students', color='tab:red')
bars2 = ax.bar(x + width/2, nonidea_rates, width, label='Non-IDEA Students', color='tab:blue')

# Labels
ax.set_ylabel('Percentage of Students', fontsize=12, fontweight='bold')
ax.set_title('Discipline Disparities: IDEA vs. Non-IDEA Students (2020-21)\nIDEA Students Face 2-3x Higher Suspension Rates', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=10)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
def autolabel(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

autolabel(bars1)
autolabel(bars2)

plt.tight_layout()
plt.show()

print("\nKey Findings:")
print("- IDEA students are 1.9x more likely to receive in-school suspensions")
print("- IDEA students are 2.5x more likely to receive out-of-school suspensions")
print("- Law enforcement referrals and arrests also disproportionately higher")
print("- Discipline disparities raise concerns about equity and appropriate behavioral supports")

## 7. Restraint and Seclusion (2020-21)

IDEA students account for 77% of all restraint and seclusion incidents, despite representing only 14% of enrollment.

In [ ]:
# Extract restraint/seclusion data
restraint = crdc_data['restraint_seclusion']
rs_summary = crdc_data['summary_statistics']['restraint_and_seclusion']

print("Restraint and Seclusion (2020-21):")
print(f"Total incidents: {rs_summary['total_incidents']:,}")
print(f"IDEA students: {rs_summary['pct_idea_of_all_rs']:.1f}% of all incidents")
print(f"\nBreakdown:")
print(f"  Physical restraint - IDEA: {rs_summary['idea_physical_restraint']:,}")
print(f"  Physical restraint - Non-IDEA: {rs_summary['nonidea_physical_restraint']:,}")
print(f"  Seclusion - IDEA: {rs_summary['idea_seclusion']:,}")
print(f"  Seclusion - Non-IDEA: {rs_summary['nonidea_seclusion']:,}")

In [ ]:
# Visualization 7: Restraint and Seclusion
fig, ax = plt.subplots(figsize=(10, 7))

# Data
rs_categories = ['Physical\nRestraint', 'Mechanical\nRestraint', 'Seclusion']
idea_rs = [
    restraint['IDEA students subjected to physical restraint']['national_total'],
    restraint['IDEA students subjected to mechanical restraint']['national_total'],
    restraint['IDEA students subjected to seclusion']['national_total']
]
nonidea_rs = [
    restraint['Non-IDEA students subjected to physical restraint']['national_total'],
    restraint['Non-IDEA students subjected to mechanical restraint']['national_total'],
    restraint['Non-IDEA students subjected to seclusion']['national_total']
]

x = np.arange(len(rs_categories))
width = 0.35

bars1 = ax.bar(x - width/2, idea_rs, width, label='IDEA Students', color='tab:orange')
bars2 = ax.bar(x + width/2, nonidea_rs, width, label='Non-IDEA Students', color='tab:cyan')

ax.set_ylabel('Number of Students', fontsize=12, fontweight='bold')
ax.set_title('Restraint and Seclusion Incidents by Student Group (2020-21)\nIDEA Students: 77% of All Incidents', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(rs_categories, fontsize=11)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

# Add value labels
def autolabel_rs(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{int(height):,}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

autolabel_rs(bars1)
autolabel_rs(bars2)

plt.tight_layout()
plt.show()

print("\nKey Findings:")
print("- IDEA students comprise 14% of enrollment but 77% of restraint/seclusion incidents")
print("- Physical restraint is the most common intervention (35,066 students total)")
print("- Seclusion used on 15,709 students (75% IDEA, 25% non-IDEA)")
print("- Raises concerns about appropriate use of restrictive interventions")

## 8. Section 504 Comparison

Section 504 serves students with disabilities who don't qualify for IDEA but need accommodations. These students represent 3.3% of enrollment (1.6 million students in 2020-21).

In [ ]:
# Extract Section 504 data
national = crdc_data['national_overview_2020_21']['total_enrollment']

print("IDEA vs. Section 504 Enrollment (2020-21):")
print(f"Total enrollment: {national['total_students']:,}")
print(f"\nIDEA students: {national['idea_students']:,} ({national['pct_idea']:.2f}%)")
print(f"Section 504 students: {national['section_504_students']:,} ({national['pct_504']:.2f}%)")
print(f"\nCombined disability services: {national['idea_students'] + national['section_504_students']:,} ({national['pct_idea'] + national['pct_504']:.2f}%)")

In [ ]:
# Visualization 8: IDEA vs Section 504 Pie Chart
fig, ax = plt.subplots(figsize=(10, 8))

sizes = [national['idea_students'], national['section_504_students'], national['non_disabled_504']]
labels = ['IDEA\n(IEP)', 'Section 504\n(504 Plan)', 'No Disability\nServices']
colors = ['#ff9999', '#ffcc99', '#99ccff']
explode = (0.05, 0.05, 0)

wedges, texts, autotexts = ax.pie(sizes, explode=explode, labels=labels, colors=colors,
                                    autopct='%1.1f%%', startangle=90, textprops={'fontsize': 11})

# Bold percentages
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

# Add counts as subtitle
ax.text(0, -1.3, f"IDEA: {national['idea_students']:,} | Section 504: {national['section_504_students']:,} | No Services: {national['non_disabled_504']:,}",
        ha='center', fontsize=10, style='italic')

ax.set_title('Public School Enrollment by Disability Service Type (2020-21)\nTotal Enrollment: 49.2 Million Students', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("\nKey Findings:")
print("- IDEA (IEP) students: 13.9% — require specialized instruction")
print("- Section 504 (504 Plan) students: 3.3% — require accommodations but not specialized instruction")
print("- Combined: 17.2% of public school students receive some form of disability services")
print("- Section 504 has grown faster than IDEA in recent years (2011: 1.5% → 2020: 3.3%)")

## Summary of Key Findings

### Enrollment
- **7.5 million students** served under IDEA in 2022-23 (15.2% of public school enrollment) — all-time high
- Enrollment doubled from **3.7M (1976)** to **7.5M (2023)** — 104% growth
- As percentage of enrollment: **8.3% (1976)** → **15.2% (2023)** — 83% relative increase

### Disability Categories
- **Specific learning disability** (32%) remains largest category, but declined from 45% in 2000
- **Autism** is fastest-growing: **1.5% (2000)** → **13% (2023)** — **8.5x increase**
- Top 3 categories (SLD, speech, other health impairment) account for **66%** of IDEA students
- **Other health impairment** (primarily ADHD) grew 3.2x since 2000

### State Variation
- Nearly **2:1 variation** between states: Pennsylvania (21.1%) vs. Hawaii (11.7%)
- Likely reflects differences in state identification practices, funding, and demographic factors
- Northeast region has highest identification rates (avg 18.6%); West region lowest (avg 14.0%)

### Discipline Disparities (CRDC 2020-21)
- IDEA students face **2-3x higher suspension rates** than non-IDEA peers
  - In-school suspension: **2.7%** (IDEA) vs. **1.4%** (non-IDEA)
  - Out-of-school suspension: **2.7%** (IDEA) vs. **1.1%** (non-IDEA)
- IDEA students account for **77% of restraint/seclusion incidents** despite being 14% of enrollment

### Section 504
- **1.6 million students** (3.3% of enrollment) served under Section 504 in 2020-21
- Section 504 has grown faster than IDEA: **1.5% (2011)** → **3.3% (2020)**
- Combined IDEA + Section 504: **17.2%** of public school students receive disability services

### Implications
- Autism explosion reflects better screening, broader diagnostic criteria, and increased awareness
- Discipline disparities suggest need for better behavioral supports and trauma-informed practices
- State variation indicates inconsistent identification practices and potential inequities
- Growth in Section 504 may reflect students with milder disabilities or those needing accommodations only